In [2]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile
import zipfile
import subprocess
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

Tensorflow version: 1.14.0


In [3]:
PROJ_DIR = f'{WORK}/ADA_Project'
RAW_IMGS_DIR = f'{PROJ_DIR}/datasets/cars_train'
RESIZED_IMGS_DIR = f'{PROJ_DIR}/datasets/cars196_resized'
DATA_CUSTOM_DIR = f'{PROJ_DIR}/datasets/cars196_custom'
TRAIN_RUNS_DIR = f'{PROJ_DIR}/training_runs/cars196_training-runs'

In [ ]:
url = 'http://imagenet.stanford.edu/internal/car196/cars_train.tgz'
urllib.request.urlretrieve(url, f'{PROJ_DIR}/datasets/cars_train.tgz')

In [ ]:
tarf = tarfile.open(f'{PROJ_DIR}/datasets/cars_train.tgz')
tarf.extractall(path=RAW_IMGS_DIR)

In [4]:
raw_imgs = glob(f'{RAW_IMGS_DIR}/**/*')
len(raw_imgs)

8144

In [5]:
for x in tqdm(raw_imgs):
    DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

100%|██████████| 8144/8144 [04:34<00:00, 29.61it/s]


In [6]:
DeepFAMS.preprocessing.tf_record_exporter(
    tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/cars196_resized"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/cars196_custom"
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/03439.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/05502.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/03469.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/00266.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/01085.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/07774.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/cars196_resized/07693.jpg
axes don't match array
Failed to export /work/chasel

In [ ]:
# # Needs to be run through the command line at least once to compile the model

# ! module load anaconda && \
#     module load compiler/gcc/4.7 && \
#     module load cuda && \
#     $WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada/train.py \
#     --outdir=$WORK/ADA_Project/training_runs/cars196_training-runs \
#     --gpus=2 \
#     --data=$WORK/ADA_Project/datasets/cars196_custom \
#     --snap=1

In [11]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ADA_Project/training_runs/cars196_training-runs/00010-cars196_custom-auto2-resumecustom/network-snapshot-000401.pkl


In [ ]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 1,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [ ]:
DeepFAMS.utils.execute('nvidia-smi')

In [ ]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)